In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
# df_sgm = pd.read_csv('df_trans_user_item_date.csv', sep='\t', dtype='object')
# df_item = pd.read_csv('MASTER_ITEM_202012121853.csv', dtype='object')
# df_sgm['TR_DATE'] = df_sgm['TR_DATE'].map(lambda x: eval(x))
# df_sgm['TR_PLU'] = df_sgm['TR_PLU'].apply(lambda x: x[0:-1])

In [3]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [4]:
# q_plu = """
# SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
# where lower(mpa.MPA_DESC) like 'sgm eksplor%'
# """

# df_plu = pd.read_sql_query(q_plu, connection)     
# li_plu = df_plu['MPA_PLU']


# df_all = []
# for plu in li_plu[0:20]:
#     df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
#     df_all.append(df)

# df_seq_1 = pd.concat(df_all)
# df_seq_1.head()
df_seq_1 = pd.DataFrame()

In [5]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like 'sgm eksplor%'
or lower(mpa.MPA_DESC) like 'anlene%'
or lower(mpa.MPA_DESC) like 'dancow%'
or lower(mpa.MPA_DESC) like 'bebelac%'
or lower(mpa.MPA_DESC) like 'lactogrow%'
or lower(mpa.MPA_DESC) like 'ff baby%'
or lower(mpa.MPA_DESC) like 'pepsodent pg economy%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_2 = pd.concat(df_all)
df_seq_2.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2021-01-05 19:01:23.420,"{'1607904000': 1, '1607558400': 1, '1603929600': 1}","{'1607904000': 1, '1607558400': 1, '1603929600': 1}",NaT,NaT
1,5fd90a51b8f8002aaaba19e7,9990011494110271,120763,2021-01-06 06:17:18.838,"{'1607904000': 1, '1597276800': 1}","{'1607904000': 1, '1597276800': 1}",NaT,NaT
2,5fd90a51b8f8002aaaba2495,9990021000743963,120763,2021-01-05 10:34:26.833,{'1607904000': 1},{'1607904000': 1},NaT,NaT
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2021-01-05 22:05:42.855,"{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}","{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2021-01-06 10:34:42.719,"{'1607904000': 1, '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1}","{'1609286400': 1, '1607904000': 1, '1603929600': 1, '1594684800': 1}",NaT,NaT


In [6]:
df_seq = pd.concat([df_seq_1, df_seq_2]).reset_index(drop=True)

In [7]:
q_nas = """
SELECT ITEM_N, DESCP, UKURAN, PANJANG, LEBAR, TINGGI, UOM FROM MASTER_NAS_M mnm 
"""

df_nas = pd.read_sql_query(q_nas, connection)
display(df_nas.head())

def norm_uom(uom):
    li_uom_norm = []
    li_weight = []
    
    if uom == np.nan:
        li_weight.append(np.nan)
        li_uom_norm.append(np.nan)
    elif uom == 'KG':
        li_weight.append(1000)
        li_uom_norm.append('G')
    elif uom == 'L':
        li_weight.append(1000)
        li_uom_norm.append('ML')
    else:
        li_weight.append(1)
        li_uom_norm.append(uom)

    return li_uom_norm[0], li_weight[0]
            
        
df_nas['UOM_NORM'], df_nas['weight'] = zip(*df_nas['UOM'].apply(norm_uom))
df_nas['UKURAN_NORM'] = df_nas['UKURAN'] * df_nas['weight']


df_nas['UOM'] = df_nas['UOM_NORM']
df_nas['UKURAN']= df_nas['UKURAN_NORM']

,ITEM_N,DESCP,UKURAN,PANJANG,LEBAR,TINGGI,UOM
0,106499,"VANDA PRG OVAL POLOS 9,5""",0.0,NaN,NaN,NaN,None
1,121493,"VANDA SENDOK SOUP POLOS 2,5""",0.0,NaN,NaN,NaN,None
2,63428,SOKLIN SOFTENER BIRU REF 450ML,450.0,6.5,13.3,19.5,ML
3,63429,SOKLIN SOFTENER KNG REF 450ML,450.0,6.0,13.8,20.0,ML
4,24102,ENERGIZER E91 BP2,1.0,1.0,1.0,1.0,BH


In [8]:
df = pd.merge(df_seq, df_nas, left_on='TR_PLU_PCS', right_on='ITEM_N')[list(df_seq) + ['DESCP','UKURAN','UOM']]
df = df[df['TRF_DATE'].notnull()]

min_freq = 6
df_hist = df[df['TRF_DATE'].apply(lambda x: len(x)) >= min_freq]

In [9]:
df_hist.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM
11,5fd90a53b8f8002aaabace0b,9990013066618077,120763,2021-01-05 13:04:09.642,"{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}","{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}",2020-12-19,2020-12-22,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
13,5fd90a54b8f8002aaabb0e0d,9990013033091757,120763,2021-01-06 06:01:16.284,"{'1607904000': 1, '1602547200': 1, '1602460800': 1, '1601942400': 1, '1597536000': 1, '1597449600': 1, '1591228800': 1, '1587859200': 1}","{'1609113600': 1, '1607904000': 1, '1602547200': 1, '1602460800': 1, '1601942400': 1, '1597536000': 1, '1597449600': 1}",2020-12-14,2021-02-06,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
14,5fd90a54b8f8002aaabb1661,9990011350125734,120763,2021-01-06 07:46:18.845,"{'1608336000': 1, '1607904000': 1, '1607558400': 1, '1606694400': 1, '1606176000': 1, '1603152000': 1, '1602892800': 1, '1601856000': 1, '1601596800': 1, '1600646400': 1, '1598400000': 1, '1596412800': 1, '1609718400': 1}","{'1609718400': 1, '1608336000': 1, '1607904000': 1, '1607558400': 1, '1606694400': 1, '1606176000': 1, '1603152000': 1, '1602892800': 1, '1601856000': 1, '1601596800': 1, '1600646400': 1, '1598400000': 1, '1596412800': 1}",2021-01-04,2021-01-17,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
25,5fd90a61b8f8002aaabc4d87,9990013096098148,120763,2021-01-05 18:07:01.957,"{'1607904000': 1, '1606176000': 1, '1606089600': 3, '1605571200': 1, '1605484800': 2, '1605312000': 1, '1605225600': 1, '1604534400': 1, '1608595200': 2, '1608508800': 1, '1608768000': 1, '1609200000': 1, '1609459200': 1, '1609718400': 1}","{'1609718400': 1, '1609459200': 1, '1609200000': 1, '1608768000': 1, '1608595200': 2, '1608508800': 1, '1607904000': 1, '1606176000': 1, '1606089600': 3, '1605571200': 1, '1605484800': 2, '1605312000': 1, '1605225600': 1, '1604534400': 1}",2021-01-04,2021-01-06,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
28,5fd90a61b8f8002aaabc89d1,9990010837218300,120763,2021-01-06 00:05:43.542,"{'1608336000': 1, '1607904000': 1, '1606608000': 1, '1606435200': 1, '1606262400': 1, '1605744000': 1, '1604966400': 1, '1604016000': 1, '1602201600': 1, '1601856000': 1, '1601164800': 1, '1600992000': 1, '1600819200': 1}","{'1608336000': 1, '1607904000': 1, '1606608000': 1, '1606435200': 1, '1606262400': 1, '1605744000': 1, '1604966400': 1, '1604016000': 1, '1602201600': 1, '1601856000': 1, '1601164800': 1, '1600992000': 1, '1600819200': 1}",2020-12-19,2020-12-20,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR


In [10]:
# set_time = {'1608163200': 1, '1607904000': 1, '1607040000': 1, '1606176000': 1, '1605916800': 1, '1605657600': 1, '1605225600': 1, '1602288000': 1}
# set_time = sorted(set_time.items())


# print(set_time)

In [11]:
def cal_feature(set_timestamp):

    set_timestamp = sorted(set_timestamp.items())
    last_order = int(set_timestamp[-1][0])
    
    li_seq = []
    for idx in range(0, len(set_timestamp)-1, 1):

        diff = int(set_timestamp[idx+1][0]) - int(set_timestamp[idx][0])
        diff_day = int((diff / 3600) / 24)
        li_seq.append(diff_day)

    read_seq = []
    qty_seq = []
    for i in set_timestamp:
        read_seq.append(datetime.fromtimestamp(int(i[0])).strftime('%d%b%y'))
        qty_seq.append(i[1])
        
    qty_seq = qty_seq[-1*(min_freq):-1]
    
    # get onyly until needed sequence order
    li_seq = li_seq[-1*(min_freq-1):]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # mean_qty
    seq_mean_qty = round(np.mean([li_seq[i]/qty_seq[i] for i in range(len(li_seq))]),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)
    
    li_seq.extend(qty_seq)
    li_seq.append(seq_mean_qty)


    return li_seq, last_order, read_seq

In [12]:
df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRF_DATE'].apply(cal_feature))

<ipython-input-12-6b00eafc0eee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRF_DATE'].apply(cal_feature))
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [13]:
#col_feature = ['t_5', 't_4', 't_3', 't_2', 't_1', 'max', 'min', 'since', 'mean', 'variance', 'q_5', 'q_4', 'q_3', 'q_2', 'q_1', 'mean_qty']
col_feature = ['t_'+ str(i) for i in range((min_freq-1),0,-1)] + \
                ['max', 'min', 'since', 'mean', 'variance'] + \
                ['q_'+ str(i) for i in range((min_freq-1),0,-1)] + ['mean_qty']
train = pd.DataFrame(df_hist.tr_date_seq.tolist(), index=df_hist.index, columns=col_feature)



In [14]:
train.head()

,t_5,t_4,t_3,t_2,t_1,max,min,since,mean,variance,q_5,q_4,q_3,q_2,q_1,mean_qty
11,7,1,1,2,3,7,1,2,2.8,4.96,1,1,1,1,1,2.8
13,51,6,1,62,14,62,1,62,26.8,617.36,1,1,1,1,1,26.8
14,6,10,4,5,16,16,4,5,8.2,19.36,1,1,1,1,1,8.2
25,1,2,5,3,3,5,1,3,2.8,1.76,1,2,1,1,1,2.6
28,6,2,2,15,5,15,2,15,6.0,22.80,1,1,1,1,1,6.0


In [15]:
# X = train_descp[list(train) + ['UKURAN']].drop(['t_1','q_1'], 1)
# Y = train_descp['t_1']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# from sklearn.linear_model import LinearRegression
# clf = LinearRegression()
# clf.fit(X_train, y_train)

# train_predictions = clf.predict(X_test)
# from sklearn import metrics
# rmse = np.sqrt( metrics.mean_squared_error( y_test, train_predictions ) )
# print("RMSE: {}".format(rmse))

In [16]:
# test = train_descp[list(train) + ['UKURAN']].drop(['t_5','q_5'], 1)

In [17]:
# test_predictions = clf.predict(test)

In [18]:
# test['next_order'] = list(test_predictions)
# test['next_order'] = test['next_order'].astype(int).abs()

In [19]:
# train

In [20]:
# clf.intercept_
# clf.coef_

### Compare time consuming in different netto
> 
* first we user with less variance, we assume user buy based on their consumption and less affected by promotion or random purchase date
* age might be affect consumption, cause more mature, amount of consumption should be decreased by parent

In [21]:
# with open('watchlist_item.txt') as f:
#     watchlist = f.read().splitlines()
#     df_watchlist = pd.DataFrame({'DESCP':watchlist})

In [22]:
train_descp = pd.concat([train,df_hist], 1)[col_feature + ['TR_MEMBERS', 'TR_PLU_PCS', 'DESCP', 'TRF_DATE', 'read_seq', 'UKURAN', 'UOM']]
train_descp = pd.merge(train_descp, df_plu[['MPA_PLU', 'MPA_DESC_SUBCAT']], left_on='TR_PLU_PCS', right_on='MPA_PLU', how='left')
train_descp = train_descp[train_descp['UKURAN'].notnull()]
train_descp['UKURAN'] = train_descp['UKURAN'].astype(int)

In [23]:
# train_descp.groupby('UKURAN').agg({'mean_qty':'mean'})

# train_descp_cond = train_descp[train_descp['variance'] < 100]
# train_descp_cond.groupby('UKURAN').agg({'mean_qty':'mean'})

In [24]:
train_descp['DESCP_TYPE'] = train_descp['DESCP'].str.split(' ').str[0:-3].str.join(' ')

In [25]:
train_descp_  = train_descp[(train_descp['mean_qty'] >= 6) & (train_descp['mean_qty'] <= 25)]

In [26]:
train_descp_.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'mean'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE       UKURAN UOM           
Perawatan Gigi & Mulut PEPSODENT        75     GR   17.813953
                       PEPSODENT PG     190    GR   17.743902
                                        225    GR   15.231346
Susu Bubuk             ANLENE ACTIFIT   250    GR   16.517658
                       ANLENE GOLD      250    GR   17.752742
                                        650    GR   18.713578
                       ANLENE GOLD+     250    GR   17.409398
                                        650    GR   18.705949
                       BEBELAC 3        200    GR   13.675781
                                        400    GR   14.197054
                                        800    GR   14.627916
                                        1000   G    15.013728
                       BEBELAC 4        400    GR   14.334203
                                        800    GR   15.219395
                                        1000   G    16.317320
                       BEBELAC GOLD 3   700    GR   13.707738
                       DANCOW 1+        200    GR   13.321470
                                        400    GR   13.796380
                                        800    GR   14.559178
                       DANCOW 3+        200    GR   13.756210
                                        400    GR   14.431024
                                        800    GR   15.332873
                       DANCOW 5+        400    GR   14.917894
                                        800    GR   15.933695
                       DANCOW FG        200    GR   15.150579
                                        400    GR   15.614557
                                        800    GR   16.012972
                       FF BABY TAHAP 1  400    GR   11.749630
                                        800    GR   11.942680
                       FF BABY TAHAP 2  400    GR   11.743284
                                        800    GR   12.174793
                       LACTOGROW 3      180    GR   13.372951
                                        350    GR   13.674159
                                        750    GR   14.252016
                       LACTOGROW 4      750    GR   14.673938
                       SGM EKSPLOR      700    GR   14.742921
                       SGM EKSPLOR 1+   150    GR   14.023456
                                        400    GR   14.310101
                                        600    GR   14.454549
                                        900    GR   14.940229
                       SGM EKSPLOR 3+   150    GR   13.638602
                                        400    GR   14.607115
                                        600    GR   14.932755
                                        900    GR   15.243695
                       SGM EKSPLOR 5+   400    GR   14.192845
                                        900    GR   15.463548
                       SGM EKSPLOR SOYA 700    GR   14.774831
Susu Cair & Es Krim    DANCOW UHT FG    110    ML   14.065823

In [27]:
train_descp_cond = train_descp[train_descp['variance'] < 80]
train_descp_cond.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'mean'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE       UKURAN UOM           
Perawatan Gigi & Mulut PEPSODENT        75     GR   16.214120
                       PEPSODENT PG     190    GR   11.820000
                                        225    GR    7.142243
Susu Bubuk             ANLENE ACTIFIT   250    GR   15.735230
                       ANLENE GOLD      250    GR   18.262069
                                        650    GR   21.388750
                       ANLENE GOLD+     250    GR   17.610351
                                        650    GR   21.165926
                       BEBELAC 3        200    GR    8.120484
                                        400    GR    9.449685
                                        800    GR   11.429036
                                        1000   G    11.884116
                       BEBELAC 4        400    GR   10.509261
                                        800    GR   12.848540
                                        1000   G    13.596595
                       BEBELAC GOLD 3   700    GR   10.854442
                       DANCOW 1+        200    GR    8.152275
                                        400    GR    9.393710
                                        800    GR   12.292299
                       DANCOW 3+        200    GR    8.960801
                                        400    GR   10.669376
                                        800    GR   13.557641
                       DANCOW 5+        400    GR   11.942950
                                        800    GR   14.731158
                       DANCOW FG        200    GR   11.079477
                                        400    GR   12.238653
                                        800    GR   13.947568
                       FF BABY TAHAP 1  400    GR    8.280670
                                        800    GR   10.375976
                       FF BABY TAHAP 2  400    GR    7.981313
                                        800    GR   10.312392
                       LACTOGROW 3      180    GR    7.445173
                                        350    GR    8.743925
                                        750    GR   11.750985
                       LACTOGROW 4      750    GR   12.507781
                       SGM EKSPLOR      700    GR   11.358186
                       SGM EKSPLOR 1+   150    GR    8.514903
                                        400    GR    9.442080
                                        600    GR   10.357333
                                        900    GR   11.648400
                       SGM EKSPLOR 3+   150    GR    8.344528
                                        400    GR   10.545317
                                        600    GR   10.969005
                                        900    GR   12.939323
                       SGM EKSPLOR 5+   400    GR   10.728932
                                        900    GR   13.460999
                       SGM EKSPLOR SOYA 700    GR   11.335693
Susu Cair & Es Krim    DANCOW UHT FG    110    ML    7.759446

In [28]:
from ipywidgets import interact, interact_manual
@interact_manual
def show_articles_more_than(nama = sorted(list(train_descp['DESCP_TYPE'].unique())), 
                            ukuran = sorted(list(train_descp['UKURAN'].unique())),
                            lower=(2, 60, 2),
                            upper=(2, 60, 2)):
    show_data = train_descp.loc[(train_descp['DESCP_TYPE'] == nama) & (train_descp['UKURAN'] == ukuran)\
                                & (train_descp['mean_qty'] >= lower) & (train_descp['mean_qty'] <= upper)]\
                .drop(['since','mean','TRF_DATE','MPA_PLU','MPA_DESC_SUBCAT','DESCP_TYPE'],1)
    print(show_data.shape)
    sample_n = 10
    if show_data.shape[0] < sample_n:
        sample_n = show_data.shape[0]
    return show_data.sample(sample_n)

interactive(children=(Dropdown(description='nama', options=('ANLENE ACTIFIT', 'ANLENE GOLD', 'ANLENE GOLD+', '…

In [29]:
all_item = pd.merge(df_plu, df_nas, left_on='MPA_PLU', right_on='ITEM_N')[['MPA_PLU', 'MPA_DESC', 'UKURAN']]
all_item = all_item.sort_values('MPA_DESC')
for idx, row in all_item.iterrows():
    
    show_data = train_descp.loc[(train_descp['TR_PLU_PCS'] == row['MPA_PLU']) & (train_descp['UKURAN'] == row['UKURAN'])]
    print(row['MPA_DESC'])
    
    #display(show_data)
    Q1 = round(show_data['mean_qty'].quantile(0.25),2)
    Q2 = round(show_data['mean_qty'].quantile(0.5),2)
    Q3 = round(show_data['mean_qty'].quantile(0.75),2)
    IQR= round(Q3 - Q1, 2)
    upper_out = Q3 + 1.5 * IQR
    lower_out = Q1 - 1.5 * IQR
    MAX = show_data['mean_qty'].max()
    MIN = show_data['mean_qty'].min()
    
    # calculate summary statistics
    data_mean, data_std = show_data['mean_qty'].mean(), show_data['mean_qty'].std()
    # identify outliers
    cut_off = data_std * 1
    lower, upper = data_mean - cut_off, data_mean + cut_off

    print("Q1:{} Q2:{} Q3:{} IQR:{} MAX:{} MIN:{}".format(Q1,Q2,Q3,IQR,MAX,MIN))
    print("lower {} upper{}".format(lower, upper))
    print()
    

ANLENE ACTIFIT COKLAT BOX 250G
Q1:14.0 Q2:19.2 Q3:25.0 IQR:11.0 MAX:35.2 MIN:1.87
lower 12.332993464730047 upper26.747397160269948

ANLENE ACTIFIT VANILA BOX 250G
Q1:nan Q2:nan Q3:nan IQR:nan MAX:nan MIN:nan
lower nan uppernan

ANLENE GOLD COKLAT BOX 250G
Q1:16.7 Q2:21.8 Q3:27.0 IQR:10.3 MAX:35.0 MIN:7.0
lower 14.747567648537665 upper28.16361514716124

ANLENE GOLD COKLAT BOX 650G
Q1:18.5 Q2:24.4 Q3:28.5 IQR:10.0 MAX:34.8 MIN:1.3
lower 16.806203609209916 upper30.064643848417234

ANLENE GOLD ORIGINAL BOX 650G
Q1:18.4 Q2:23.2 Q3:29.0 IQR:10.6 MAX:34.2 MIN:3.7
lower 16.446565996721695 upper29.660352922197234

ANLENE GOLD PLUS VANILA BOX250G
Q1:15.8 Q2:21.8 Q3:27.2 IQR:11.4 MAX:35.2 MIN:2.9
lower 14.17344030358786 upper28.65897528082774

ANLENE GOLD PLUS VANILA BOX650G
Q1:18.8 Q2:24.2 Q3:28.2 IQR:9.4 MAX:35.4 MIN:1.0
lower 16.634427383560634 upper29.66570913521069

ANLENE UHT COKELAT TP 175ML
Q1:nan Q2:nan Q3:nan IQR:nan MAX:nan MIN:nan
lower nan uppernan

ANLENE UHT VANILA TP 175ML
Q1:nan 

Q1:11.6 Q2:16.6 Q3:22.3 IQR:10.7 MAX:35.4 MIN:0.68
lower 10.028245745298122 upper24.357756567732075

SGM EKSPLOR 5+ MADU BOX 400G
Q1:9.2 Q2:13.85 Q3:20.0 IQR:10.8 MAX:35.0 MIN:0.35
lower 7.793593033879299 upper22.153832708694907

SGM EKSPLOR 5+ MADU BOX 900G
Q1:11.4 Q2:16.6 Q3:22.4 IQR:11.0 MAX:35.6 MIN:0.1
lower 9.695555051343996 upper24.473360806261198

SGM EKSPLOR SOYA MADU BOX 700G
Q1:10.4 Q2:15.4 Q3:21.0 IQR:10.6 MAX:35.8 MIN:0.49
lower 8.899518028276209 upper23.391085118576928

SGM EKSPLOR SOYA VANILA BOX700G
Q1:10.4 Q2:15.2 Q3:21.4 IQR:11.0 MAX:35.8 MIN:0.76
lower 8.866639939152082 upper23.481676643762533

